# Libbitcoin BX: Transactions with absolute locktime.
In this BX tutorial, we will send funds to a `checklocktimeverify` encumbered output, and spend this after the absolute locktime has expired.
<img src="images/cltv_overview.jpg" alt="drawing" style="" width="700px"/>

<hr style="border: 0.5px dashed #000;">

## 1) Setup wallet and test funds
### 1.1) Generate private/public key pair
#### 1.1.1) New private key

In [ ]:
bx seed | bx ec-new

#### 1.1.2) Save your private key

In [ ]:
echo e60c1ee5fb938aa041c15594b0df6ffbedbfc33e4aaf231b7eb4dbf18c464d8e

#### 1.1.3) Derive public key<br>

In [ ]:
bx ec-to-public e60c1ee5fb938aa041c15594b0df6ffbedbfc33e4aaf231b7eb4dbf18c464d8e

#### 1.1.4) Derive public key hash

In [ ]:
bx sha256 02e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31 | bx ripemd160

### 1.2) Derive testnet address
Add version prefix for mainnet/testnet<br>
* `6f` in hex
* `111` in decimal

And base58 encode result

In [ ]:
bx wrap-encode -v 111 42a8bcd5f61fa17adbe800de6382f24bd1db0d7f \
| bx base58-encode


### 1.3) Send test coins funds to your testnet address
Note your spendable p2pkh UXTO's below...

* UXTO 0: 
    * TXID: `423f3180d4a0f95c925553beef5e2fec7a4f19f9632c8294575ac450ff5b6b89`<br>
    * Index: `0`
    * Amount: `130000000`

* UXTO 0: 
    * TXID: `d4552abb5edd8ce6eb8a8399d717f6636356e8a8d6242d7f50832399a7ab238e`<br>
    * Index: `0`
    * Amount: `64998700`

<hr style="border: 0.5px dashed #000;">

## 2) Spending to a CLTV encumbered output<br>
<img src="images/spend_to_cltv.jpg" alt="drawing" style="" width="700px"/>

### 2.1) Build transaction with CLTV encumbered output for signing

#### 2.1.1) Determine absolute locktime in blockheight
* `$ bx fetch-height`
* `$ expr [current blockheight] + [delay in blocks]`.
* Encode locktime in hex (little endian)

In [ ]:
bx fetch-height 

In [ ]:
expr 1412027 + 4

In [ ]:
printf '%x\n' 1412016

**Note:** Don't forget to flip byte-order (4-Bytes) above for output script below.

#### 2.1.2) Encode CLTV-encumbered P2PKH output script
* **`[locktime]`**
* **`CHECKLOCKTIMEVERIFY`**
* `DROP`
* `DUP`
* `HASH160`
* `[public key hash]`
* `EQUALVERIFY`
* `CHECKSIG`

In [ ]:
bx script-encode \
"[b08b1500] CHECKLOCKTIMEVERIFY DROP DUP HASH160 [42a8bcd5f61fa17adbe800de6382f24bd1db0d7f] EQUALVERIFY CHECKSIG"


#### 2.1.3) Calculate output amount

* `400 Satoshi/sigops + 1 Satoshi/Byte` 
* Output amount(s) are previous output amount(s) minus fees

In [2]:
expr 130000000 - 400 - 250 - 1000


129998350


#### 2.1.4) Encode transaction for signing
* `bx tx-encode -i [prevout txid:index:sequence] -o [output script:amount]`
* Input sequence defaults to `ffffffff`/`4294967295`

In [ ]:
bx tx-encode \
--input 423f3180d4a0f95c925553beef5e2fec7a4f19f9632c8294575ac450ff5b6b89:0:4294967295 \
--output 04b08b1500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac:129998350


### 2.2) Building the input script

#### 2.2.1) Signing the transaction with the private key
* `bx fetch-tx` previous output script
* `bx input-sign` `[private key]` `["previous output script"]` `[encoded transaction]`
    * Outputs DER encoded signature

In [ ]:
bx fetch-tx -f json 423f3180d4a0f95c925553beef5e2fec7a4f19f9632c8294575ac450ff5b6b89 \
| jq ".transaction.outputs[0].script"

In [ ]:
bx input-sign \
e60c1ee5fb938aa041c15594b0df6ffbedbfc33e4aaf231b7eb4dbf18c464d8e \
"dup hash160 [42a8bcd5f61fa17adbe800de6382f24bd1db0d7f] equalverify checksig" \
0100000001896b5bff50c45a5794822c63f9194f7aec2f5eefbe5355925cf9a0d480313f420000000000ffffffff010e9ebf07000000002004b08b1500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000


#### 2.2.2) Set the input script into the finalised transaction
* The input script to spend the previous p2pkh output script is `[endorsement]` `[public key]`
* `bx input-set` `"input script"` `[encoded transaction]`

In [ ]:
bx input-set \
"[3044022020ff2e82115e6822c0d69b0c4370dba42de61ad24c650563242125758258d5e902201af2906a9f0b74f28f90378303896798a0d70077770f9ba455ecc5613178b68e01] [02e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31]" \
0100000001896b5bff50c45a5794822c63f9194f7aec2f5eefbe5355925cf9a0d480313f420000000000ffffffff010e9ebf07000000002004b08b1500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000


### 2.3) Validate & Broadcast the endorsed transaction
* `bx validate-tx` `[encoded transaction]`
* `bx tx-decode -f json` `[encoded transaction]` | `jq ".transaction.hash"`

In [ ]:
bx validate-tx \
0100000001896b5bff50c45a5794822c63f9194f7aec2f5eefbe5355925cf9a0d480313f42000000006a473044022020ff2e82115e6822c0d69b0c4370dba42de61ad24c650563242125758258d5e902201af2906a9f0b74f28f90378303896798a0d70077770f9ba455ecc5613178b68e012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31ffffffff010e9ebf07000000002004b08b1500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000


In [ ]:
bx tx-decode -f json \
0100000001896b5bff50c45a5794822c63f9194f7aec2f5eefbe5355925cf9a0d480313f42000000006a473044022020ff2e82115e6822c0d69b0c4370dba42de61ad24c650563242125758258d5e902201af2906a9f0b74f28f90378303896798a0d70077770f9ba455ecc5613178b68e012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31ffffffff010e9ebf07000000002004b08b1500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000 \
|jq ".transaction.hash"

In [ ]:
bx send-tx 0100000001896b5bff50c45a5794822c63f9194f7aec2f5eefbe5355925cf9a0d480313f42000000006a473044022020ff2e82115e6822c0d69b0c4370dba42de61ad24c650563242125758258d5e902201af2906a9f0b74f28f90378303896798a0d70077770f9ba455ecc5613178b68e012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31ffffffff010e9ebf07000000002004b08b1500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000


<hr style="border: 0.5px dashed #000;">

## 3) Spend the CLTV encumbered UXTO

<img src="images/spend_from_cltv.jpg" alt="drawing" style="" width="700px"/>

### 3.1) Build transaction with activated locktime for signing

#### 3.1.1) Encode p2pkh output script

In [ ]:
bx script-encode "DUP HASH160 [42a8bcd5f61fa17adbe800de6382f24bd1db0d7f] EQUALVERIFY CHECKSIG"

**3.1.2) Encode transaction with locktime & set input sequence**<br>
* Reference Previous Outpoint<br>
* Transaction locktime field must be `>=` locktime in CLTV output being spent.
* Input sequence msut be less than maximum, so `fffffffe`/`4294967294`

In [ ]:
expr 129998350 - 400 - 250 - 1000

In [ ]:
bx tx-encode \
--lock_time 1411982 \
--input e25ac53e48592030908ce9e438f6fd93dd01364c7562d5646a4373d67ddcbb7a:0:4294967294 \
--output 76a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac:129996700


### 3.2) Building the input script

#### 3.2.1)  Signing the transaction with the private key
* `bx fetch-tx` previous output script
* `bx input-sign` `[private key]` `["previous output script"]` `[encoded transaction]`
    * Outputs DER encoded signature

In [ ]:
bx fetch-tx -f json e25ac53e48592030908ce9e438f6fd93dd01364c7562d5646a4373d67ddcbb7a \
| jq ".transaction.outputs[0].script"


In [ ]:
bx input-sign e60c1ee5fb938aa041c15594b0df6ffbedbfc33e4aaf231b7eb4dbf18c464d8e \
"[b08b1500] CHECKLOCKTIMEVERIFY DROP DUP HASH160 [42a8bcd5f61fa17adbe800de6382f24bd1db0d7f] EQUALVERIFY CHECKSIG" \
01000000017abbdc7dd673436a64d562754c3601dd93fdf638e4e98c90302059483ec55ae20000000000feffffff019c97bf07000000001976a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac8e8b1500


#### 3.2.2) Set the input script into the finalised transaction

In [ ]:
bx input-set "[3045022100e99a3e961b76a91bafeca92fd5e3f514873c1c5439debfe3ff573c6aef351f87022008ccf9f779c5542464d8e2ca932fbba482b134ce0f2fe644c8efb89a0d6326a301] [02e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31]" \
01000000017abbdc7dd673436a64d562754c3601dd93fdf638e4e98c90302059483ec55ae20000000000feffffff019c97bf07000000001976a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac8e8b1500


### 3.3) Validate & Broadcast the endorsed transaction
* `bx validate-tx` `[encoded transaction]`
* Get TXID: `bx tx-decode -f json` `[encoded transaction]` | `jq ".transaction.hash"`
* `bx send-tx` `[encoded transaction]`

In [ ]:
bx validate-tx \
01000000017abbdc7dd673436a64d562754c3601dd93fdf638e4e98c90302059483ec55ae2000000006b483045022100e99a3e961b76a91bafeca92fd5e3f514873c1c5439debfe3ff573c6aef351f87022008ccf9f779c5542464d8e2ca932fbba482b134ce0f2fe644c8efb89a0d6326a3012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31feffffff019c97bf07000000001976a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac8e8b1500


In [ ]:
bx tx-decode -f json \
01000000017abbdc7dd673436a64d562754c3601dd93fdf638e4e98c90302059483ec55ae2000000006b483045022100e99a3e961b76a91bafeca92fd5e3f514873c1c5439debfe3ff573c6aef351f87022008ccf9f779c5542464d8e2ca932fbba482b134ce0f2fe644c8efb89a0d6326a3012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31feffffff019c97bf07000000001976a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac8e8b1500 \
| jq ".transaction.hash"
